In [2]:
import pandas as pd
import spacy

In [3]:
nlp = spacy.load('ru_core_news_sm')

In [4]:
data = pd.read_csv('RBK.csv')

In [10]:
tokenized_data = []
for index, row in data.iterrows():
    title = row['Название']
    text = row['Текст']
    
    title_tokens = [token.text for token in nlp(title)]
    text_tokens = [token.text for token in nlp(text)]

    
    tokenized_data.append((title_tokens, text_tokens))

new_data = pd.DataFrame(tokenized_data, columns=['Название (токены)', 'Текст (токены)'])


new_data.to_csv('tokenf.csv', index=False)

In [11]:
from transformers import BertTokenizer, BertForSequenceClassification

model_name = 'DeepPavlov/rubert-base-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

In [13]:
new_data = pd.read_csv('tokenf.csv')

In [14]:
title_ids = []
text_ids = []

In [15]:
for index, row in data.iterrows():
    title_tokens = row['Название (токены)']
    text_tokens = row['Текст (токены)']
    
    title_input = tokenizer.encode_plus(title_tokens)['input_ids']
    text_input = tokenizer.encode_plus(text_tokens)['input_ids']
    
    title_ids.append(title_input)
    text_ids.append(text_input)
    
tokenized_data = pd.DataFrame({'Название (идентификаторы)': title_ids, 'Текст (идентификаторы)': text_ids})
tokenized_data.to_csv('tokenized_data.csv', index=False)


In [23]:
import torch
from torch.utils.data import Dataset, DataLoader

class BERTDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = self.data[index]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask
        }
tokenized_data = pd.read_csv('tokenized_data.csv')

# Создание экземпляра класса BERTDataset
max_length = 256
dataset = BERTDataset(tokenized_data['Название (идентификаторы)'], tokenizer, max_length)

# Создание загрузчика данных
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [24]:
for batch in dataloader:
    input_ids = batch['input_ids'].to('cuda') # Перемещение данных на устройство (например, GPU)
    attention_mask = batch['attention_mask'].to('cuda') # Перемещение данных на устройство (например, GPU)

    # Выполнение обучения или предсказаний с использованием модели BERT
    outputs = model(input_ids, attention_mask)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx